In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
from sklearn import preprocessing 
from datetime import datetime

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

In [ ]:
# kfold crossval, train-test and hold-out

In [ ]:
# Lasso regression (to determine relevant features)
# Cox survival analysis
# Supervised classifiers & ensemble
# Tensorflow

In [ ]:
# gridsearch
# comparison of models
# various metrics

In [ ]:
# conclusions about best model
# visualization of results